In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
import tensorflow as tf

[nltk_data] Downloading package stopwords to /Users/Tian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_ui = pd.read_csv('eclipse_platform_ui_no_space.csv')

corpus = []
labels = []

In [3]:

df_ui.head()


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,bug_id,summary,description,bag_of_word_stemmed,summary_stemmed,description_stemmed,report_time,report_timestamp,status,commit,commit_timestamp,files,files_no_space
0,0,0,0,1,384108,Bug 384108 JUnit view icon no longer shows pr...,Build Identifier: Version: Juno Release Build ...,unit junit view icon longer show progress exec...,unit junit view icon longer show progress exec...,build identifi version juno releas build id up...,2012-07-03 03:39:25,1.341300e+09,resolved fixed,5da5952,1.389970e+09,bundles/org.eclipse.e4.ui.workbench/src/org/ec...,bundles/org.eclipse.e4.ui.workbench/src/org/ec...
1,1,1,1,2,385394,Bug 385394 Performance issue regarding enable...,Build Identifier: Version: 4.2.0 Build id: I20...,perform issu regard enabl state handl menu con...,perform issu regard enabl state handl menu con...,build identifi version build id i20120608 hi c...,2012-07-18 07:41:06,1.342610e+09,resolved fixed,8db6c32,1.389890e+09,bundles/org.eclipse.e4.ui.workbench.renderers....,bundles/org.eclipse.e4.ui.workbench.renderers....
2,2,2,2,3,423588,Bug 423588 [QuickAccess] Quick Access failure...,I was working in an inner from master.\n\nWhen...,quick access quickaccess quick access failur d...,quick access quickaccess quick access failur d...,work inner master restart workspac editor open...,2013-12-09 08:48:44,1.386600e+09,resolved fixed,6750d25,1.389820e+09,bundles/org.eclipse.ui.workbench/Eclipse UI/or...,bundles/org.eclipse.ui.workbench/EclipseUI/org...
3,3,3,3,4,420238,Bug 420238 [CSS] Reduce whitespace usage in t...,One of the major complains I hear is that the ...,cs reduc whitespac usag default theme one majo...,cs reduc whitespac usag default theme,one major complain hear new style wast lot spa...,2013-10-24 04:08:35,1.382600e+09,verified fixed,7710840,1.389800e+09,bundles/org.eclipse.e4.ui.workbench.renderers....,bundles/org.eclipse.e4.ui.workbench.renderers....
4,4,4,4,5,405216,Bug 405216 Widget disposed exception in Progr...,I have a job that sets the IProgressConstants....,widget dispos except progress info item progre...,widget dispos except progress info item progre...,job set progress constant iprogressconst actio...,2013-04-08 18:21:10,1.365460e+09,resolved fixed,470f274,1.389720e+09,bundles/org.eclipse.ui.workbench/Eclipse UI/or...,bundles/org.eclipse.ui.workbench/EclipseUI/org...


In [4]:
def sentence_to_stemmed(sentence):
    sentence = sentence.lower()
    sentence = sentence.split()
    porterStemmer = PorterStemmer()
    sentence = [porterStemmer.stem(word) for word in sentence if not word in set(stopwords.words('english'))]
    sentence = ' '.join(sentence)

    return sentence

In [5]:
rows, cols = df_ui.shape

In [6]:
for i in range(rows):
    report = re.sub('[^a-zA-Z0-9]', ' ', (df_ui['summary'][i]))
    label = df_ui['files'][i]
    corpus.append(sentence_to_stemmed(report))
    label = re.sub('\.', '_', label)
    label = re.sub('/', '__', label)
    labels.append(label)
    if i % 100 == 0:
        print('%d lines processed.\n' % i)

0 lines processed.

100 lines processed.

200 lines processed.

300 lines processed.

400 lines processed.

500 lines processed.

600 lines processed.

700 lines processed.

800 lines processed.

900 lines processed.

1000 lines processed.

1100 lines processed.

1200 lines processed.

1300 lines processed.

1400 lines processed.

1500 lines processed.

1600 lines processed.

1700 lines processed.

1800 lines processed.

1900 lines processed.

2000 lines processed.

2100 lines processed.

2200 lines processed.

2300 lines processed.

2400 lines processed.

2500 lines processed.

2600 lines processed.

2700 lines processed.

2800 lines processed.

2900 lines processed.

3000 lines processed.

3100 lines processed.

3200 lines processed.

3300 lines processed.

3400 lines processed.

3500 lines processed.

3600 lines processed.

3700 lines processed.

3800 lines processed.

3900 lines processed.

4000 lines processed.

4100 lines processed.

4200 lines processed.

4300 lines processed.



In [7]:
countVectorizer = CountVectorizer()

X = countVectorizer.fit_transform(corpus).toarray()
y = countVectorizer.fit_transform(labels).toarray()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =0)

In [9]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [10]:
num_filters = 128
l2_reg_lambda = 0.03
dropout_keep_prob = 0.5
batch_size = 256
num_epochs = 100
evaluate_every = 100
embedding_dim = 128
filter_sizes = "3,4,5"

In [ ]:
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) 
    with sess.as_default():
	cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda)	

        ## Define Training procedure
        train_op = tf.train.AdamOptimizer(1e-3).minimize(cnn.loss)

        ## Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        ## Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        ## Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)


        ## Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch, step):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob
            }
            _, summaries, loss, accuracy = sess.run(
                [train_op, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
	    
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, step, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            summaries, loss, accuracy = sess.run(
                [dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), batch_size, num_epochs)
        # Training loop. For each batch...
	current_step = 0
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch, current_step)
            current_step += 1 
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, current_step, writer=dev_summary_writer)
                print("")